In [9]:
# import Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("IntervalSets")
# Pkg.add("LatexStrings")

In [10]:
using DataFrames, CSV
using IntervalArithmetic
using LaTeXStrings
using Statistics
# using Base
include("./libs/display-latex.jl")
include("./libs/crisp-pcm.jl")
include("./libs/analysis-indicators.jl")
include("./libs/solve-deterministic-ahp.jl")
include("./libs/MMR.jl")

MMR (generic function with 1 method)

In [11]:
# 評価基準数
num_criteria = [4,5,6,7,8];
# 真の区間重要度のセッティング
true_importance = ["A", "B", "C", "D", "E"]; # ["A", "B", "C", "D", "E"]

In [12]:
# 真の区間重要度を読み込み
for N in num_criteria
        
    for setting in true_importance
        df = CSV.File("./data/N"*string(N)*"_"*setting*"_Given_interval_weight.csv", header=false) |> DataFrame
        T = Vector{Interval{Float64}}() # Float64のところにTを使うとおかしくなるのはなぜか？
        for i in 1:N
            push!(T, interval(df[1, 2*i-1], df[1, 2*i]))
        end

        # display(L"T = %$(intervalVectorLaTeXString(T))")
        # 一対比較行列を1000個読み込み
        df1 = CSV.File("./data/N"*string(N)*"_a3_"*setting*"_PCM_int.csv", header=false) |> DataFrame

        # データフレームをN×Nのサブデータフレームに分割
        subdfs = split_dataframe(df1, N);
        # 結果格納用
        P = Matrix{Float64}(undef, N, 1000)
        Q = Matrix{Float64}(undef, N, 1000)
        R = Matrix{Float64}(undef, N, 1000)
        F = Matrix{Float64}(undef, N, 1000)

        # 1000個のデータで推定
        for (i, subdf) in enumerate(subdfs)

            # 一対比較行列を取り出す
            A = Matrix(subdf)
            # display(L"A = %$(matrixLaTeXString(A))")
            # if i == 779
            #     println("i = ", i)
            #     display(L"A = %$(matrixLaTeXString(A))")
            # end

            # 提案手法で区間重要度推定
            solution = MMR(A, EV)
            E = solution.W # 区間重要度の推定値
            # display(L"E = %$(intervalVectorLaTeXString(E))")
            # display(L"T = %$(intervalVectorLaTeXString(T))")

            TcapE = T .∩ E # 真の区間重要度と推定値の積集合
            TcupE = T .∪ E # 真の区間重要度と推定値の和集合
            # display(L"T ∩ E = %$(intervalVectorLaTeXString(TcapE))")
            # display(L"T ∪ E = %$(intervalVectorLaTeXString(TcupE))")
            
            # P, Q, R, F の計算
            P[:, i] = calculate_P(T, E) # 真の区間重要度と推定値の積集合の割合
            Q[:, i] = calculate_Q(T, E) # 真の区間重要度と推定値の積集合の割合
            R[:, i] = calculate_R(T, E) # 真の区間重要度と推定値の積集合の割合
            F[:, i] = calculate_F(T, E) # F値
            # println("i = ", i)
            # display(L"P = %$(VectorLaTeXString(P[:,i]))")
            # display(L"Q = %$(VectorLaTeXString(Q[:,i]))")
            # display(L"R = %$(VectorLaTeXString(R[:,i]))")
            # display(L"F = %$(VectorLaTeXString(F[:,i]))")
        end

        println("N = ", N)
        println("setting = ", setting)

        P_ave = mean(P)
        Q_ave = mean(Q)
        R_ave = mean(R)
        F_ave = mean(F)

        display(L"Pₐᵥₑ = %$(round(P_ave, digits=5))")
        display(L"Qₐᵥₑ = %$(round(Q_ave, digits=5))")
        display(L"Rₐᵥₑ = %$(round(R_ave, digits=5))")
        display(L"Fₐᵥₑ = %$(round(F_ave, digits=5))")
        # P_var = var(P)
        # Q_var = var(Q)
        # R_var = var(R)
        # F_var = var(F)
        # display(L"Pᵥₐᵣ = %$(round(P_var, digits=5))")
        # display(L"Qᵥₐᵣ = %$(round(Q_var, digits=5))")
        # display(L"Rᵥₐᵣ = %$(round(R_var, digits=5))")
        # display(L"Fᵥₐᵣ = %$(round(F_var, digits=5))")

    end
end


L"$Pₐᵥₑ = 0.46181$"

L"$Qₐᵥₑ = 0.79239$"

L"$Rₐᵥₑ = 0.5498$"

L"$Fₐᵥₑ = 0.5972$"

N = 4
setting = A


...

L"$Pₐᵥₑ = 0.54356$"

L"$Qₐᵥₑ = 0.99649$"

L"$Rₐᵥₑ = 0.54523$"

L"$Fₐᵥₑ = 0.69608$"

N = 8
setting = E
